# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import pickle
import string
import random
from collections import defaultdict
import cv2
from keras.utils import np_utils, to_categorical

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
from keras.applications import mobilenet_v2
from keras.preprocessing import image

def prepare_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = mobilenet_v2.preprocess_input(x)
    return x

# Data Entry

In [3]:
# organize bird images to be classified
types_list = ['catbirds','crows','hummingbirds','ravens','sparrows','terns','woodpeckers','wrens']
root_folder = '../../Desktop/birdup/birds_by_type/'
bird_directories = []
directory_catbirds = root_folder + types_list[0] + '/'
bird_directories.append(directory_catbirds)
directory_crows = root_folder + types_list[1] + '/'
bird_directories.append(directory_crows)
directory_hummingbirds = root_folder + types_list[2] + '/'
bird_directories.append(directory_hummingbirds)
directory_ravens = root_folder + types_list[3] + '/'
bird_directories.append(directory_ravens)
directory_sparrows = root_folder + types_list[4] + '/'
bird_directories.append(directory_sparrows)
directory_terns = root_folder + types_list[5] + '/'
bird_directories.append(directory_terns)
directory_woodpeckers = root_folder + types_list[6] + '/'
bird_directories.append(directory_woodpeckers)
directory_wrens = root_folder + types_list[7] + '/'
bird_directories.append(directory_wrens)


y = []
x = []

image_dict = defaultdict(list)

#add images and labels for each species to x and y variables
for idx, directory in enumerate(bird_directories):

    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            filename = directory + filename
            image_dict[directory].append(prepare_image(filename).reshape(224,224,3))
            x.append(prepare_image(filename).reshape(224,224,3))
            y.append(idx)
        
x = np.array(x)
y = np.array(y)

In [4]:
X_train, X_test, y_train, y_test = (train_test_split(x, y, 
                                                     test_size = .2, random_state = 25))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
base_model = mobilenet_v2.MobileNetV2(weights='imagenet',
                                      include_top=False, input_shape=(224,224,3)
                                     ) 
 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x) # flatten from convolution tensor output 
x = Dense(100, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(50, activation='relu')(x)
predictions = Dense(8, activation='softmax')(x) # should match # of classes predicted

# define formal model object to train and compile it as usual
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit(X_train, y_train)

print(accuracy_score(np.argmax(y_train,axis=-1), np.argmax(model.predict(X_train), axis=-1)))
print(accuracy_score(np.argmax(y_test,axis=-1), np.argmax(model.predict(X_test), axis=-1)))

46/46 [==============================] - 28s 613ms/step - loss: 1.1785 - accuracy: 0.7439
0.8872950819672131
0.8555858310626703


In [ ]:
import streamlit as st 
from PIL import Image
from classify import predict
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

st.title("Sort Your Bird Images")

uploaded_file = st.file_uploader("Choose an image...", type="jpg")
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Classifying...")
    label = predict(uploaded_file)
    st.write('%s (%.2f%%)' % (label[1], label[2]*100))


In [42]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

def predict(image_path):
    
    y_labels = {0: 'Dark_eyed_Junco',
             1: 'Brown_Thrasher',
             2: 'Common_Tern',
             3: 'Chipping_Sparrow',
             4: 'American_Crow',
             5: 'Pileated_Woodpecker',
             6: 'American_Goldfinch',
             7: 'Red_bellied_Woodpecker',
             8: 'Red_winged_Blackbird',
             9: 'Cardinal',
             10: 'Great_Crested_Flycatcher',
             11: 'Forsters_Tern',
             12: 'Common_Raven',
             13: 'House_Wren',
             14: 'Belted_Kingfisher',
             15: 'Cedar_Waxwing',
             16: 'White_breasted_Nuthatch',
             17: 'House_Sparrow',
             18: 'Scarlet_Tanager',
             19: 'Red_headed_Woodpecker',
             20: 'Downy_Woodpecker',
             21: 'Indigo_Bunting',
             22: 'Warbling_Vireo',
             23: 'Rose_breasted_Grosbeak',
             24: 'Barn_Swallow',
             25: 'Ring_billed_Gull',
             26: 'Gray_Catbird',
             27: 'Song_Sparrow',
             28: 'Northern_Flicker',
             29: 'Baltimore_Oriole',
             30: 'Blue_Jay',
             31: 'Tree_Swallow',
             32: 'White_crowned_Sparrow',
             33: 'Rock_Wren',
             34: 'Ruby_throated_Hummingbird',
             35: 'Mallard'}
    
    #load previously trained and saved model
    model = keras.models.load_model('first_streamlit_model/')
    
    #convert jpg file to numpy array for modeling
    image = np.array(prepare_image(image_path).reshape(224,224,3))
    image = np.expand_dims(image, axis=0)

    # predict the probability across all output classes
    yhat = model.predict(image)
    
    # return species name and probability for the most likely class
    label = y_labels[np.argmax(yhat[0])]
    label_prob = sorted(list(yhat[0]))[-1]
    return (label, label_prob) 

In [1]:
from keras.applications import mobilenet_v2
from keras.preprocessing import image

def prepare_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = mobilenet_v2.preprocess_input(x)
    return x

In [44]:
predict('data/CUB_200_2011/images/010.Red_winged_Blackbird/Red_Winged_Blackbird_0020_4050.jpg')

('Red_winged_Blackbird', 1.0)

In [2]:
img = image.load_img('data/CUB_200_2011/images/010.Red_winged_Blackbird/Red_Winged_Blackbird_0020_4050.jpg', target_size=(224, 224))

In [3]:
type(img)

PIL.Image.Image